Setting the env variables

In [1]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

Setting the GROQ LLM Instance

In [2]:
if not os.environ['GROQ_API_KEY']:
    os.environ['GROQ_API_KEY'] = getpass.getpass("Enter API key for GROQ: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

Setting the Embedding Instance

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    cache_folder="./models"
)

c:\Users\Asus\.conda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the vector store instance

In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings_dim = len(embeddings.embed_query("hello worls"))
index = faiss.IndexFlatL2(embeddings_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={},
)

Build an app that answers questions about the website's content
Website - https://lilianweng.github.io/posts/2023-06-23-agent/

In [5]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

#Load and chunk contents of the blob
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
all_splits = text_splitter.split_documents(docs)

#index chunks
_ = vector_store.add_documents(documents=all_splits)

prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    answer: str
    context: List[Document]

def retrieve(state: State):
    retrived_docs = vector_store.similarity_search(state["question"])
    return {
        "context": retrived_docs,
    }

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {
        "answer": response.content,
    }

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Trigger the Agent

In [6]:
response = graph.invoke({
    "question": "What is Task Decomposition?",
})

print(response["answer"])

Task Decomposition is a process that breaks down a complex task into smaller, manageable subtasks. This can be done using various techniques, such as simple prompting, task-specific instructions, or human inputs. The goal is to transform big tasks into multiple smaller tasks, making it easier for an agent or model to plan and execute the task.
